In [20]:
# Install the required libraries.
!pip install tensorflow opencv-contrib-python youtube-dl moviepy pydot

In [21]:
!pip install git+https://github.com/TahaAnwar/pafy.git#egg=pafy

  Cloning https://github.com/TahaAnwar/pafy.git to c:\users\ashis\appdata\local\temp\pip-install-tatvl1tb\pafy_5fdee0239a66417787838613542ed926


  Running command git clone --filter=blob:none --quiet https://github.com/TahaAnwar/pafy.git 'C:\Users\ashis\AppData\Local\Temp\pip-install-tatvl1tb\pafy_5fdee0239a66417787838613542ed926'
  fatal: unable to access 'https://github.com/TahaAnwar/pafy.git/': Failed to connect to github.com port 443 after 21077 ms: Timed out
  error: subprocess-exited-with-error
  
  git clone --filter=blob:none --quiet https://github.com/TahaAnwar/pafy.git 'C:\Users\ashis\AppData\Local\Temp\pip-install-tatvl1tb\pafy_5fdee0239a66417787838613542ed926' did not run successfully.
  exit code: 128
  
  See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

git clone --filter=blob:none --quiet https://github.com/TahaAnwar/pafy.git 'C:\Users\ashis\AppData\Local\Temp\pip-install-tatvl1tb\pafy_5fdee0239a66417787838613542ed926' did not run successfully.
exit code: 128

See above for output.

note: This error originates from

In [22]:
# Import the required libraries.
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [23]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [24]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 20

# Specify the directory containing the UCF50 dataset. 
DATASET_DIR = "SPHAR-Dataset-1.0/videos"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = [name for name in os.listdir('SPHAR-Dataset-1.0/videos') if os.path.isdir(os.path.join('SPHAR-Dataset-1.0/videos', name))]

In [26]:
CLASSES_LIST

['carcrash',
 'falling',
 'hitting',
 'igniting',
 'kicking',
 'luggage',
 'murdering',
 'panicking',
 'running',
 'sitting',
 'stealing',
 'vandalizing']

In [27]:
def frames_extraction(video_path):

    # Declare a list to store video frames.
    frames_list = []
    
    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video. 
        success, frame = video_reader.read() 

        # Check if Video frame is not successfully read then break the loop
        if not success:
            break

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        
        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)
    
    # Release the VideoCapture object. 
    video_reader.release()

    # Return the frames list.
    return frames_list

In [28]:
def create_dataset():
    
    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):
        
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        i=0
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            if(i>20):
                break
            i=i+1
            
#             print(f'Extracting Data of '+file_name)
            
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  
    
    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

Now we will utilize the function **`create_dataset()`** created above to extract the data of the selected classes and create the required dataset.

In [29]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: carcrash
Extracting Data of Class: falling
Extracting Data of Class: hitting
Extracting Data of Class: igniting
Extracting Data of Class: kicking
Extracting Data of Class: luggage
Extracting Data of Class: murdering
Extracting Data of Class: panicking
Extracting Data of Class: running
Extracting Data of Class: sitting
Extracting Data of Class: stealing
Extracting Data of Class: vandalizing


In [30]:
len(features)

234

In [31]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [32]:
# Split the Data into Train ( 75% ) and Test Set ( 25% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.2, shuffle = True,
                                                                            random_state = seed_constant)

In [33]:
def create_convlstm_model():
    model = Sequential()
    
    model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (SEQUENCE_LENGTH,
                                                                                      IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    
    model.add(Flatten()) 
    
    model.add(Dense(len(CLASSES_LIST), activation = "softmax"))
    
    model.summary()
    
    return model

In [34]:
# Construct the required convlstm model.
convlstm_model = create_convlstm_model()

# Display the success message. 
print("Model Created Successfully!")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_4 (ConvLSTM2D)  (None, 20, 62, 62, 4)     1024      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 20, 31, 31, 4)    0         
 3D)                                                             
                                                                 
 time_distributed_3 (TimeDis  (None, 20, 31, 31, 4)    0         
 tributed)                                                       
                                                                 
 conv_lstm2d_5 (ConvLSTM2D)  (None, 20, 29, 29, 8)     3488      
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 20, 15, 15, 8)    0         
 3D)                                                             
                                                      

In [35]:
# Plot the structure of the contructed model.
plot_model(convlstm_model, to_file = 'convlstm_model_structure_plot.png', show_shapes = True, show_layer_names = True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [36]:
# Compile the model and specify loss function, optimizer and metrics values to the model
convlstm_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start training the model.
convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs = 55, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2)

Epoch 1/55
38/38 [==============================] - 56s 1s/step - loss: 2.4832 - accuracy: 0.1007 - val_loss: 2.5368 - val_accuracy: 0.0263
Epoch 2/55
38/38 [==============================] - 42s 1s/step - loss: 2.4442 - accuracy: 0.1275 - val_loss: 2.4391 - val_accuracy: 0.1053
Epoch 3/55
38/38 [==============================] - 43s 1s/step - loss: 2.2586 - accuracy: 0.2013 - val_loss: 2.2473 - val_accuracy: 0.1316
Epoch 4/55
38/38 [==============================] - 46s 1s/step - loss: 1.9379 - accuracy: 0.2752 - val_loss: 1.8943 - val_accuracy: 0.3158
Epoch 5/55
38/38 [==============================] - 47s 1s/step - loss: 1.7174 - accuracy: 0.3960 - val_loss: 1.7678 - val_accuracy: 0.3421
Epoch 6/55
38/38 [==============================] - 55s 1s/step - loss: 1.5129 - accuracy: 0.4497 - val_loss: 1.6173 - val_accuracy: 0.3684
Epoch 7/55
38/38 [==============================] - 50s 1s/step - loss: 1.4126 - accuracy: 0.4497 - val_loss: 1.4901 - val_accuracy: 0.3158
Epoch 8/55
38/38 [==

In [37]:
# Evaluate the trained model.
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)

2/2 [==============================] - 2s 598ms/step - loss: 3.3539 - accuracy: 0.4681


In [38]:
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

model_file_name = f'convlstm_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

convlstm_model.save(model_file_name)

In [39]:
features, labels, video_files_paths

features = np.array(features)
np.save('features.npy',features)

labels = np.array(labels)
np.save('labels.npy',labels)

with open('video_files_paths.txt', 'w') as f:
    for item in video_files_paths:
        f.write('%s\n' % item)